# TPGs selection

In [ ]:
import numpy as np
import pandas as pd
import json

arr = np.load(r"./disease gene reprioritization/result/logits202.npy")
rp = arr[174].tolist()   #disease index  #uc 191 #unipolar 192  asd 2  # 174 schizophrenia

scz_dict = dict(zip(range(18547),rp))

or_scz_dict = sorted(scz_dict.items(), key= lambda d:d[1], reverse= True)
print(or_scz_dict[:300])
selected_L = or_scz_dict[:300]


#entrez to symbol
fw = open(r"./disease gene reprioritization/result/graphene_scz_300_logits.txt", "w")
df = pd.read_csv(r"./network_data/sy_ent_ens.csv")  #symbol,entrez,ensg of gene file

#node json map

fj =open(r"./network_data/map.json",'r')
json_map = json.load(fj)

map_dict = json_map[0]


for item in selected_L:
    gene2 = df[df["entrez_id"].astype(str) == map_dict[str(item[0])]]["symbol"].values
    if gene2.size > 0:
        gene = str(gene2[0])
        fw.write(gene + "\n")

fw.close()

# biological process annotation

In [ ]:
import re
import pandas as pd

fr = open(r"./GO_Biological_Process_2021_300.txt")

fr_gene = open(r"./disease gene reprioritization/result/graphene_scz_300_logits.txt")

ll = fr.readlines()
ll_gene = fr_gene.readlines()
df_list = []
for i in range(1,len(ll_gene)):
    go_list_tmp = []
    gene = ll_gene[i].strip()
    for j in range(len(ll)):
        line = re.split("\t",ll[j].strip())
        annotation = re.findall(r"[(](.*?)[)]", re.split("\t",ll[j].strip())[1])
        if gene in re.split(";", line[-1]):
            go_list_tmp.append(annotation[0])
    if go_list_tmp:
        go_list = ";".join(go_list_tmp)
        df_list.append({"gene":gene,"go_annotation":go_list})
df1 = pd.DataFrame(df_list)
df1.to_csv(r"./scz_logits_300_annot_bp.csv", index = False)

# biological process annotation classify

In [3]:
import json
import os
import re
import pandas as pd

df1 = pd.read_csv(r"./scz_logits_300_annot_bp.csv")
df = pd.read_csv(r"./go_tree.csv")

ger_list = ["GO:0010468","GO:0032774","GO:0051252"]
nc_list = ["GO:0099536","GO:0007154","GO:0023052","GO:0005737","GO:0007267","GO:0042391"]
cyt_list = ["GO:0070507","GO:0032886","GO:0000226","GO:0007010","GO:0006996"]
nsd_p_list = ["GO:0048854","GO:0009887","GO:0007399","GO:0050877"]
cca_list = ["GO:0098742","GO:0098609","GO:0007155"]
ion_list = ["GO:0006811","GO:0006810"]
gly_sac_lipid_list = ["GO:0009311","GO:0070085","GO:0009100","GO:0009101","GO:0005976" \
                      ,"GO:0006024","GO:0044255","GO:1901657","GO:1901657","GO:1903509","GO:0006664"]
class_list = []
for i in range(len(df1)):
    line = df1.iloc[i]["go_annotation"]
    go_list = re.split(";", line)
    flag = 0
    for go_term in go_list:
        if go_term in ger_list:
            df1.loc[i,"class"] = "ger"
            flag = 1
            break
        ancestors = df[df["go_annot"].astype(str) == go_term]["go_list"].values
        if ancestors.size > 0:
            ancestors = re.split(";", ancestors[0])
            for single in ancestors:
                if single in ger_list:
                    df1.loc[i,"class"] = "ger"
                    #df1.at[0,'age']=25
                    flag = 1
                    break
        if flag == 1:
            break
    if flag == 0:
        for go_term in go_list:
            if go_term in nc_list:
                df1.loc[i,"class"] = "nc"
                flag = 1
                break
            ancestors = df[df["go_annot"].astype(str) == go_term]["go_list"].values
            if ancestors.size > 0:
                ancestors = re.split(";", ancestors[0])
                for single in ancestors:
                    if single in nc_list:
                        df1.loc[i,"class"] = "nc"
                        flag = 1
                        break
            if flag == 1:
                break
    if flag == 0:
        for go_term in go_list:
            if go_term in cyt_list:
                df1.loc[i,"class"] = "cyt"
                flag = 1
                break
            ancestors = df[df["go_annot"].astype(str) == go_term]["go_list"].values
            if ancestors.size > 0:
                ancestors = re.split(";", ancestors[0])
                for single in ancestors:
                    if single in cyt_list:
                        df1.loc[i,"class"] = "cyt"
                        flag = 1
                        break
            if flag == 1:
                break
    if flag == 0:
        for go_term in go_list:
            if go_term in nsd_p_list:
                df1.loc[i,"class"] = "nsd"
                flag = 1
                break
            ancestors = df[df["go_annot"].astype(str) == go_term]["go_list"].values
            if ancestors.size > 0:
                ancestors = re.split(";", ancestors[0])
                for single in ancestors:
                    if single in nsd_p_list:
                        df1.loc[i,"class"] = "nsd"
                        flag = 1
                        break
            if flag == 1:
                break
    if flag == 0:
        for go_term in go_list:
            if go_term in cca_list:
                df1.loc[i,"class"] = "cca"
                flag = 1
                break
            ancestors = df[df["go_annot"].astype(str) == go_term]["go_list"].values
            if ancestors.size > 0:
                ancestors = re.split(";", ancestors[0])
                for single in ancestors:
                    if single in cca_list:
                        df1.loc[i,"class"] = "cca"
                        flag = 1
                        break
            if flag == 1:
                break
    if flag == 0:
        for go_term in go_list:
            if go_term in ion_list:
                df1.loc[i,"class"] = "ion"
                flag = 1
                break
            ancestors = df[df["go_annot"].astype(str) == go_term]["go_list"].values
            if ancestors.size > 0:
                ancestors = re.split(";", ancestors[0])
                for single in ancestors:
                    if single in ion_list:
                        df1.loc[i,"class"] = "ion"
                        flag = 1
                        break
            if flag == 1:
                break
    if flag == 0:
        for go_term in go_list:
            if go_term in gly_sac_lipid_list:
                df1.loc[i,"class"] = "gly"
                flag = 1
                break
            ancestors = df[df["go_annot"].astype(str) == go_term]["go_list"].values
            if ancestors.size > 0:
                ancestors = re.split(";", ancestors[0])
                for single in ancestors:
                    if single in gly_sac_lipid_list:
                        df1.loc[i,"class"] = "gly"
                        flag = 1
                        break
            if flag == 1:
                break
        
df1.to_csv(r"./scz_logits_300_annot_bp_class.csv", index = False)